In [37]:

import pandas as pd
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [38]:
# set target positions for a spread of 25, 50, 75, 100, etc...

# get data

In [39]:
DAY = 0
df = pd.read_csv(f"./round-3-island-data-bottle/prices_round_3_day_{DAY}.csv", sep=";", header=0)

In [40]:
def swmid(row):
    bid = row['bid_price_1']
    ask = row['ask_price_1']
    bid_volume = row['bid_volume_1']
    ask_volume = row['ask_volume_1']
    return ((bid*ask_volume + ask*bid_volume)/(bid_volume + ask_volume))

def mm_mid_basket(row, volume_cutoff=10):
    # Find the best bid with volume >= volume_cutoff
    for i in range(1, 4):
        if row[f'bid_volume_{i}'] >= volume_cutoff:
            best_bid = row[f'bid_price_{i}']
            break
    else:
        # No bid with sufficient volume found
        best_bid = None

    # Find the best ask with volume >= volume_cutoff
    for i in range(1, 4):
        if row[f'ask_volume_{i}'] >= volume_cutoff:
            best_ask = row[f'ask_price_{i}']
            break
    else:
        # No ask with sufficient volume found
        best_ask = None

    # Calculate the mid-price if both best bid and best ask are found
    if best_bid is not None and best_ask is not None:
        mid_price = (best_bid + best_ask) / 2
        return mid_price
    else:
        # Return the mid_price column value as default
        return row['mid_price']
    
    

def fair_price(row):
    if row['product'] == 'GIFT_BASKET':
        return mm_mid_basket(row, volume_cutoff=10)
    else:
        return swmid(row)


In [41]:

df['fair'] = df.apply(fair_price, axis=1)

In [42]:
# Define the weights dictionary
weights = {
    'CHOCOLATE': 4,
    'STRAWBERRIES': 6,
    'ROSES': 1
}

# Get the unique product names
products = df['product'].unique()

# Create a new dataframe 'df_fairs' with the desired columns
columns = ['timestamp'] + list(products) + ['SYNTHETIC']
df_fairs = pd.DataFrame(columns=columns)

# Iterate over unique timestamps in the original dataframe
for timestamp in df['timestamp'].unique():
    # Get the rows for the current timestamp
    rows = df[df['timestamp'] == timestamp]
    
    # Create a dictionary to store the fair values for each product
    fairs = {}
    
    # Iterate over each product and extract its fair value
    for product in products:
        fair = rows.loc[rows['product'] == product, 'fair'].values[0]
        fairs[product] = fair
    
    # Calculate the synthetic fair
    synthetic_fair = sum(fairs[product] * weights.get(product, 0) for product in ['CHOCOLATE', 'STRAWBERRIES', 'ROSES'])
    
    # Create a new row as a DataFrame
    new_row = pd.DataFrame({'timestamp': [timestamp], **{product: [fairs[product]] for product in products}, 'SYNTHETIC': [synthetic_fair]})
    
    # Concatenate the new row with df_fairs
    df_fairs = pd.concat([df_fairs, new_row], ignore_index=True)

# Reset the index of df_fairs (optional)
df_fairs = df_fairs.reset_index(drop=True)

/var/folders/qd/7_02v1xx1nx_p2rj9mxjqzzh0000gn/T/ipykernel_19907/630176225.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fairs = pd.concat([df_fairs, new_row], ignore_index=True)


In [43]:
df = df_fairs

# binning target positions

In [44]:
df_fairs = df.copy()

In [45]:
df_fairs['SPREAD'] = df_fairs['GIFT_BASKET'] - df_fairs['SYNTHETIC'] - 370

In [46]:
df_fairs

,timestamp,CHOCOLATE,STRAWBERRIES,ROSES,GIFT_BASKET,SYNTHETIC,SPREAD
0,0,8000.000000,4000.000000,15000.000000,71355.0,71000.000000,-15.000000
1,100,8001.261307,3999.500000,15000.000000,71349.5,71002.045226,-22.545226
2,200,8001.000000,3999.500000,15000.743902,71344.5,71001.743902,-27.243902
3,300,8002.255814,3999.500000,15000.000000,71350.0,71006.023256,-26.023256
4,400,8002.000000,3999.227011,14997.500000,71341.5,71000.862069,-29.362069
...,...,...,...,...,...,...,...
9995,999500,7987.310881,4031.000000,14543.231707,71141.5,70678.475231,93.024769
9996,999600,7988.708571,4031.000000,14548.000000,71137.5,70688.834286,78.665714
9997,999700,7987.718750,4031.000000,14546.277108,71128.5,70683.152108,75.347892
9998,999800,7989.500000,4031.000000,14546.755814,71138.0,70690.755814,77.244186


In [47]:
bins = [-np.inf, -150, -100,-75, -50,-25, 25, 50, 75, 100, 150, np.inf]
labels = [60,45, 30, 20, 10, 0, -10, -20, -30,-45, -60]

# Create the "POSITION" column based on the conditions
df_fairs['POSITION'] = pd.cut(df_fairs['SPREAD'], bins=bins, labels=labels, right=False)

# Convert the "POSITION" column to numeric type
df_fairs['POSITION'] = pd.to_numeric(df_fairs['POSITION'])

In [48]:
df_fairs

,timestamp,CHOCOLATE,STRAWBERRIES,ROSES,GIFT_BASKET,SYNTHETIC,SPREAD,POSITION
0,0,8000.000000,4000.000000,15000.000000,71355.0,71000.000000,-15.000000,0
1,100,8001.261307,3999.500000,15000.000000,71349.5,71002.045226,-22.545226,0
2,200,8001.000000,3999.500000,15000.743902,71344.5,71001.743902,-27.243902,10
3,300,8002.255814,3999.500000,15000.000000,71350.0,71006.023256,-26.023256,10
4,400,8002.000000,3999.227011,14997.500000,71341.5,71000.862069,-29.362069,10
...,...,...,...,...,...,...,...,...
9995,999500,7987.310881,4031.000000,14543.231707,71141.5,70678.475231,93.024769,-30
9996,999600,7988.708571,4031.000000,14548.000000,71137.5,70688.834286,78.665714,-30
9997,999700,7987.718750,4031.000000,14546.277108,71128.5,70683.152108,75.347892,-30
9998,999800,7989.500000,4031.000000,14546.755814,71138.0,70690.755814,77.244186,-30


In [49]:
df_fairs['POSITION_DIFF'] = df_fairs['POSITION'].diff()

# Create a new column "TRADE" to mark the trades
df_fairs['TRADE'] = np.where(df_fairs['POSITION_DIFF'] != 0, df_fairs['POSITION_DIFF'], 0)

# Create a new column "CASH_CHANGE" to calculate the cash change for each trade
df_fairs['CASH_CHANGE'] = -df_fairs['TRADE'] * df_fairs['SPREAD']

# Create a new column "CASH" to track the cumulative cash position
df_fairs['CASH'] = df_fairs['CASH_CHANGE'].cumsum()

In [50]:
df_fairs

,timestamp,CHOCOLATE,STRAWBERRIES,ROSES,GIFT_BASKET,SYNTHETIC,SPREAD,POSITION,POSITION_DIFF,TRADE,CASH_CHANGE,CASH
0,0,8000.000000,4000.000000,15000.000000,71355.0,71000.000000,-15.000000,0,NaN,NaN,NaN,NaN
1,100,8001.261307,3999.500000,15000.000000,71349.5,71002.045226,-22.545226,0,0.0,0.0,0.000000,0.000000
2,200,8001.000000,3999.500000,15000.743902,71344.5,71001.743902,-27.243902,10,10.0,10.0,272.439024,272.439024
3,300,8002.255814,3999.500000,15000.000000,71350.0,71006.023256,-26.023256,10,0.0,0.0,0.000000,272.439024
4,400,8002.000000,3999.227011,14997.500000,71341.5,71000.862069,-29.362069,10,0.0,0.0,0.000000,272.439024
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,999500,7987.310881,4031.000000,14543.231707,71141.5,70678.475231,93.024769,-30,0.0,0.0,-0.000000,58547.816045
9996,999600,7988.708571,4031.000000,14548.000000,71137.5,70688.834286,78.665714,-30,0.0,0.0,-0.000000,58547.816045
9997,999700,7987.718750,4031.000000,14546.277108,71128.5,70683.152108,75.347892,-30,0.0,0.0,-0.000000,58547.816045
9998,999800,7989.500000,4031.000000,14546.755814,71138.0,70690.755814,77.244186,-30,0.0,0.0,-0.000000,58547.816045


In [51]:
# lagged returns from n timesteps negative when spread is positive -> sell signal
# lagged returns from n timesteps positive when spread is negative -> buy signal

# trading with ewma fair

In [52]:
df_fairs = df.copy()

In [53]:
df_fairs

,timestamp,CHOCOLATE,STRAWBERRIES,ROSES,GIFT_BASKET,SYNTHETIC
0,0,8000.000000,4000.000000,15000.000000,71355.0,71000.000000
1,100,8001.261307,3999.500000,15000.000000,71349.5,71002.045226
2,200,8001.000000,3999.500000,15000.743902,71344.5,71001.743902
3,300,8002.255814,3999.500000,15000.000000,71350.0,71006.023256
4,400,8002.000000,3999.227011,14997.500000,71341.5,71000.862069
...,...,...,...,...,...,...
9995,999500,7987.310881,4031.000000,14543.231707,71141.5,70678.475231
9996,999600,7988.708571,4031.000000,14548.000000,71137.5,70688.834286
9997,999700,7987.718750,4031.000000,14546.277108,71128.5,70683.152108
9998,999800,7989.500000,4031.000000,14546.755814,71138.0,70690.755814


In [54]:
df_fairs['BASKET_MINUS_SYNTHETIC'] = df_fairs['GIFT_BASKET'] - df_fairs["SYNTHETIC"]

In [55]:
alpha = 0.1
df_fairs['BASKET_MINUS_SYNTHETIC_EMA'] = df_fairs['BASKET_MINUS_SYNTHETIC'].ewm(alpha=alpha, adjust=False).mean()
df_fairs['BASKET_MINUS_SYNTHETIC_MINUS_EMA'] = df_fairs['BASKET_MINUS_SYNTHETIC'] - df_fairs['BASKET_MINUS_SYNTHETIC_EMA']

In [56]:
df_fairs

,timestamp,CHOCOLATE,STRAWBERRIES,ROSES,GIFT_BASKET,SYNTHETIC,BASKET_MINUS_SYNTHETIC,BASKET_MINUS_SYNTHETIC_EMA,BASKET_MINUS_SYNTHETIC_MINUS_EMA
0,0,8000.000000,4000.000000,15000.000000,71355.0,71000.000000,355.000000,355.000000,0.000000
1,100,8001.261307,3999.500000,15000.000000,71349.5,71002.045226,347.454774,354.245477,-6.790704
2,200,8001.000000,3999.500000,15000.743902,71344.5,71001.743902,342.756098,353.096539,-10.340442
3,300,8002.255814,3999.500000,15000.000000,71350.0,71006.023256,343.976744,352.184560,-8.207816
4,400,8002.000000,3999.227011,14997.500000,71341.5,71000.862069,340.637931,351.029897,-10.391966
...,...,...,...,...,...,...,...,...,...
9995,999500,7987.310881,4031.000000,14543.231707,71141.5,70678.475231,463.024769,473.373446,-10.348677
9996,999600,7988.708571,4031.000000,14548.000000,71137.5,70688.834286,448.665714,470.902673,-22.236959
9997,999700,7987.718750,4031.000000,14546.277108,71128.5,70683.152108,445.347892,468.347195,-22.999303
9998,999800,7989.500000,4031.000000,14546.755814,71138.0,70690.755814,447.244186,466.236894,-18.992708
